In [26]:
import pandas as pd
import plotly.express as px
import math

In [27]:
df = pd.read_csv('captura.csv')

print(df.to_string()) 

   ('BROADCAST', 34969)  ('UNICAST', 2048)  ('UNICAST', 34525)  ('BROADCAST', 2054)  ('UNICAST', 2054)  ('BROADCAST', 34999)  ('BROADCAST', 2048)
0                 329.0             9364.0               144.0                108.0               26.0                   6.0                 23.0


In [28]:
# Hago una lista de los nombres de las columnas del dataset
nombresDeSimbolos = []
for col in df.columns:
    nombresDeSimbolos.append(col)

In [29]:
# Cuento cuantas apariciones hay según el tipo de comunicación
segunDestinatario = [["UNICAST", "BROADCAST"],[0 , 0]]

for i in range(len(nombresDeSimbolos)):
    if ("UNICAST" in nombresDeSimbolos[i]):
        segunDestinatario[1][0] = segunDestinatario[1][0] + df.iat[0,i]
    if ("BROADCAST" in nombresDeSimbolos[i]):        
        segunDestinatario[1][1] = segunDestinatario[1][1] + df.iat[0,i]

# Hago el gráfico
destinatario = segunDestinatario[0]
cantidad = segunDestinatario[1]

fig = px.bar(
    x= destinatario, 
    y= cantidad,
    labels={
            "x": "Tipo de comunicación",
            "y": "Cantidad de veces censado",
                    },
    height= 400,
    width= 400,
    text_auto=True)
fig.show()

In [30]:
# Cuento cuantas apariciones hay según el protocolo
segunProtocolo = [["2048", "34525", "34969", "2054", "34999"],[0, 0, 0, 0, 0]]

for i in range(len(nombresDeSimbolos)):
    if ("2048" in nombresDeSimbolos[i]):
        segunProtocolo[1][0] = segunProtocolo[1][0] + df.iat[0,i]
    if ("34525" in nombresDeSimbolos[i]):
        segunProtocolo[1][1] = segunProtocolo[1][1] + df.iat[0,i]
    if ("34969" in nombresDeSimbolos[i]):
        segunProtocolo[1][2] = segunProtocolo[1][2] + df.iat[0,i]
    if ("2054" in nombresDeSimbolos[i]):
        segunProtocolo[1][3] = segunProtocolo[1][3] + df.iat[0,i]
    if ("34999" in nombresDeSimbolos[i]):
        segunProtocolo[1][4] = segunProtocolo[1][4] + df.iat[0,i]

#Hago el gráfico
destinatario = segunProtocolo[0]
cantidad = segunProtocolo[1]
  
fig = px.bar(
    x= destinatario, 
    y= cantidad,
    labels={
            "x": "Protocolo usado",
            "y": "Cantidad de veces censado",
                    },
    height= 400,
    width= 400,
    text_auto=True)
fig.show()

In [31]:
cantAparicionesPorSimbolo = df.values.tolist()[0]

probabilidadPorSimbolo = [cantApariciones/10000 for cantApariciones in cantAparicionesPorSimbolo]
informacionPorSimbolo = [-math.log2(probabilidad) for probabilidad in probabilidadPorSimbolo]

entropia = sum([probabilidadPorSimbolo[simbolo] * informacionPorSimbolo[simbolo] for simbolo in range(len(nombresDeSimbolos))])

In [32]:
#Hago el gráfico
  
fig = px.bar(
    y= list(reversed(nombresDeSimbolos)), 
    x= informacionPorSimbolo,
    labels={
            "y": "Símbolo de la fuente",
            "x": "Información por símbolo",
                    },
    height= 500,
    width= 700,
    text_auto=True,
    orientation='h')
fig.add_vline(x=entropia)
fig.show()